In [ ]:
!pip install -q kaggle
!pip install -q opencv-python
!pip install -q pydot
!pip install -q graphviz
!pip install -q torchview

In [ ]:
# Upload kaggle.json file
# from google.colab import files
# files.upload()

# Remove file if exist
# !rm ~/.kaggle/kaggle.json

# # Create a kaggle directory
# !mkdir -p ~/.kaggle

# # Copy the kaggle.json file to kaggle directory
# !cp kaggle.json ~/.kaggle

# # Fix permissions
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle datasets download -d msambare/fer2013
!unzip -o fer2013.zip

In [ ]:
import os
import pandas as pd

# Define a function to caculate the number of each emotion classes
train_dir = './train/'
test_dir = './test/'

def Classes_Count( path, name):
    Classes_Dict = {}

    for Class in os.listdir(path):

        Full_Path = os.path.join(path, Class)
        Classes_Dict[Class] = len(os.listdir(Full_Path))

    df = pd.DataFrame(Classes_Dict, index=[name])

    return df

Train_Count = Classes_Count(train_dir, 'Train').transpose().sort_values(by="Train", ascending=False)
Test_Count = Classes_Count(test_dir, 'Test').transpose().sort_values(by="Test", ascending=False)

pd.concat([Train_Count,Test_Count] , axis=1)

In [ ]:
# Show sample images
import matplotlib.pyplot as plt
import cv2

plt.style.use('default')
plt.figure(figsize = (25, 8))
image_count = 1
BASE_URL = './train/'

for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL +'/'+ directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(1, 7, image_count)
                image_count += 1
                image = cv2.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(directory, fontsize = 20)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


image_generator = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

training_data = image_generator.flow_from_directory(train_dir,
                                             target_size=(48,48),
                                             batch_size=32,
                                             color_mode = "grayscale",
                                             class_mode = "categorical")

test_data = image_generator.flow_from_directory(test_dir,
                                          target_size=(48,48),
                                          batch_size=32,
                                          color_mode='grayscale',
                                          class_mode='categorical')

# CNN Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(filters = 256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
# Convolutional Layer 2
model.add(Conv2D(filters = 128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
# Convolutional Layer 3
model.add(Conv2D(filters = 128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
# Convolutional Layer 4
model.add(Conv2D(filters = 64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
## Full connection layer
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
# Output Layer
model.add(Dense(7, activation='softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
## Training the model
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
history = model.fit(training_data,
                    epochs=50,
                    batch_size=32,
                    validation_data = test_data,
                    shuffle=True,
                    callbacks = [early_stop]
                   )

In [ ]:
fig, ax = plt.subplots(1, 2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12, 4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')
 
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')
 
plt.show()

In [ ]:
loss, accu = model.evaluate(test_data)

print("Loss on test set: ", loss)
print("Accuracy on test set: ", accu)

In [ ]:
model.save('fer_2013.h5')
model.save_weights('fer_2013_weights.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
test_labels = test_data.classes
predictions = model.predict(test_data)
predicted_labels = predictions.argmax(axis=1)
cm = confusion_matrix(test_labels, predicted_labels)

plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=emotion_labels, yticklabels=emotion_labels)
plt.xlabel('Predicted')
plt.ylabel('True Label')
plt.show()